# Standard CNN Architectures
* https://keras.io/applications/
* The 9 Deep Learning Papers You Need To Know About: https://adeshpande3.github.io/adeshpande3.github.io/The-9-Deep-Learning-Papers-You-Need-To-Know-About.html

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import matplotlib.pylab as plt
import numpy as np

In [4]:
from distutils.version import StrictVersion

In [5]:
import sklearn
print(sklearn.__version__)

assert StrictVersion(sklearn.__version__ ) >= StrictVersion('0.18.1')

0.18.1


In [6]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

assert StrictVersion(tf.__version__) >= StrictVersion('1.1.0')

1.2.1


In [7]:
import keras
print(keras.__version__)

assert StrictVersion(keras.__version__) >= StrictVersion('2.0.0')

Using TensorFlow backend.


2.0.8


In [8]:
import pandas as pd
print(pd.__version__)

assert StrictVersion(pd.__version__) >= StrictVersion('0.20.0')

0.20.1


## Preparation

In [9]:
# for VGG, ResNet, and MobileNet
# INPUT_SHAPE = (224, 224)

# for InceptionV3, InceptionResNetV2, Xception
INPUT_SHAPE = (299, 299)

In [10]:
EPOCHS = 50

In [11]:
# Depends on harware GPU architecture, set as high as possible (this works well on K80)
BATCH_SIZE = 500

In [12]:
!rm -rf ./tf_log
# https://keras.io/callbacks/#tensorboard
tb_callback = keras.callbacks.TensorBoard(log_dir='./tf_log')
# To start tensorboard
# tensorboard --logdir=./tf_log
# open http://localhost:6006

In [13]:
!ls -lh

total 316M
-rw-rw-r-- 1 ubuntu ubuntu  44K Oct  1 08:04 440px-Beagle_Upsy.jpg
drwxrwxr-x 8 ubuntu ubuntu 4.0K Oct  1 08:10 augmented-signs
-rw-rw-r-- 1 ubuntu ubuntu  17M Oct  1 08:10 augmented-signs.zip
-rw-rw-r-- 1 ubuntu ubuntu 303K Sep 27 15:22 Black_New_York_stuy_town_squirrel_amanda_ernlund.jpeg
-rw-rw-r-- 1 ubuntu ubuntu 844K Oct  1 08:04 cat-bonkers.png
-rw-rw-r-- 1 ubuntu ubuntu 140K Sep 27 15:22 cnn-augmentation.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 1.6M Oct  1 08:04 cnn-comparing-all-models.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 137K Oct  1 10:23 cnn-imagenet-retrain.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 158K Oct  1 08:04 cnn-intro.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 1.3M Oct  1 08:04 cnn-prediction.ipynb
-rw-rw-r-- 1 ubuntu ubuntu  88K Oct  1 10:36 cnn-standard-architectures.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 198K Oct  1 08:04 cnn-train-augmented.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 495K Sep 27 15:22 london.jpg
drwxrwxr-x 3 ubuntu ubuntu 4.0K Sep 27 15:25 __MACOSX
-rw-rw-r-- 1 ubuntu ubuntu 127K Se

In [14]:
import os
import skimage.data
import skimage.transform
from keras.utils.np_utils import to_categorical
import numpy as np

def load_data(data_dir, type=".ppm"):
    num_categories = 6

    # Get all subdirectories of data_dir. Each represents a label.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
    labels = []
    images = []
    for d in directories:
        label_dir = os.path.join(data_dir, d)
        file_names = [os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith(type)]
        # For each label, load it's images and add them to the images list.
        # And add the label number (i.e. directory name) to the labels list.
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(int(d))
    images64 = [skimage.transform.resize(image, INPUT_SHAPE) for image in images]
    y = np.array(labels)
    y = to_categorical(y, num_categories)
    X = np.array(images64)
    return X, y

In [15]:
# Load datasets.
ROOT_PATH = "./"
original_dir = os.path.join(ROOT_PATH, "speed-limit-signs")
original_images, original_labels = load_data(original_dir, type=".ppm")

In [16]:
X, y = original_images, original_labels

### Uncomment next three cells if you want to train on augmented image set
#### Otherwise Overfitting can not be avoided because image set is simply too small

In [17]:
# !curl -O https://raw.githubusercontent.com/DJCordhose/speed-limit-signs/master/data/augmented-signs.zip
# from zipfile import ZipFile
# zip = ZipFile('augmented-signs.zip')
# zip.extractall('.')

In [18]:
data_dir = os.path.join(ROOT_PATH, "augmented-signs")
augmented_images, augmented_labels = load_data(data_dir, type=".png")

In [19]:
# merge both data sets

all_images = np.vstack((X, augmented_images))
all_labels = np.vstack((y, augmented_labels))

# shuffle
# https://stackoverflow.com/a/4602224

p = numpy.random.permutation(len(all_labels))
shuffled_images = all_images[p]
shuffled_labels = all_labels[p]
X, y = shuffled_images, shuffled_labels

### Split test and train data 80% to 20%

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, y_train.shape

((3335, 299, 299, 3), (3335, 6))

# Training Xception
* Slighly optimized version of Inception: https://keras.io/applications/#xception
* Inception V3 no longer using non-sequential tower architecture, rahter short cuts: https://keras.io/applications/#inceptionv3
* Uses Batch Normalization: https://keras.io/layers/normalization/#batchnormalization 
  * Batch Normalization still exist even in prediction model
  * normalizes activations for each batch around 0 and standard deviation close to 1
  * replaces Dropout except for final fc layers
    * as a next step might make sense to alter classifier to again have Dropout for training
* All that makes it ideal for our use case

![Inception V3](https://djcordhose.github.io/ai/img/architectures/inception_v3_architecture.png)

In [21]:
from keras.applications.xception import Xception

model = Xception(classes=6, weights=None)

In [22]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, None, None, 32 128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, None, None, 32 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [23]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
!rm -rf tf_log

OSError: [Errno 12] Cannot allocate memory

## This is a truly complex model
### Batch size needs to be small overthise model does not fit in memory
### Will take long to train, even on GPU
### on augmented dataset 4 minutes on K80 per Epoch: 400 Minutes for 100 Epochs = 6-7 hours

In [27]:
# Depends on harware GPU architecture, model is really complex, batch needs to be small (this works well on K80)
BATCH_SIZE = 30

In [28]:
# %time model.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=[tb_callback], batch_size=BATCH_SIZE)
%time model.fit(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[tb_callback], batch_size=BATCH_SIZE)

Train on 2668 samples, validate on 667 samples
Epoch 1/100
2668/2668 [==============================] - 230s - loss: 1.7417 - acc: 0.3025 - val_loss: 1.7900 - val_acc: 0.2144
Epoch 2/100
2668/2668 [==============================] - 230s - loss: 1.5824 - acc: 0.3542 - val_loss: 1.7874 - val_acc: 0.2204
Epoch 3/100
2540/2668 [===========================>..] - ETA: 10s - loss: 1.4988 - acc: 0.3957

KeyboardInterrupt: 

## X for validation

## TODO: Metrics for Augmented Data

### Accuracy
![Accuracy Xception](https://djcordhose.github.io/ai/img/tensorboard/cnn-acc-xception.png)
### Validation Accuracy
![Validation Accuracy Xception](https://djcordhose.github.io/ai/img/tensorboard/cnn-val-acc-xception.png)

In [29]:
train_loss, train_accuracy = model.evaluate(X_train, y_train, batch_size=BATCH_SIZE)
train_loss, train_accuracy

3335/3335 [==============================] - 82s    


(1.7148642078868632, 0.31454273523285292)

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
test_loss, test_accuracy

In [25]:
original_loss, original_accuracy = model.evaluate(original_images, original_labels, batch_size=BATCH_SIZE)
original_loss, original_accuracy

ResourceExhaustedError: OOM when allocating tensor with shape[379,147,147,128]
	 [[Node: block2_sepconv1/separable_conv2d = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](block2_sepconv1/separable_conv2d/depthwise, block2_sepconv1/pointwise_kernel/read)]]
	 [[Node: metrics/acc/Mean/_95 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_3435_metrics/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'block2_sepconv1/separable_conv2d', defined at:
  File "/home/ubuntu/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/lib/python3.6